In [12]:
import pandas as pd
import os
import json
import ffmpeg
import imageio_ffmpeg

In [13]:
name_log = "1 - Preparations.txt"
path_out = "/home/elton/Projetos/Translater_Video/teste/resultado_teste"

In [14]:
texto = ''
with open(os.path.join(path_out, name_log), 'r', encoding='utf-8') as file:
    texto = file.read()
    texto = json.loads(texto)
    

In [15]:
lista = []

In [16]:
for i in texto['segments']:
    lista.append({
        'id': i['id'],
        'text': i['text'],
        'start': round(i['start'], 2),
        'end': round(i['end'], 2)
    })

In [26]:
lista_2 = lista

In [17]:
name_audio = "1 - Preparations.aac"

In [ ]:
def picotar_audio (path_in:str, path_out:str, name_audio:str, log:list):
    input = os.path.join(path_in, name_audio);
    path_ffmpeg = imageio_ffmpeg.get_ffmpeg_exe()
    
    for i in log:
        start_audio = i['start']
        end_audio = i['end']
        output = os.path.join(path_out, ( str(i['id']) + '.wav'))
        
        #ss -> Inicio, to -> Fim, ac -> mono, ar -> sample_rate, sample_fmt -> bit 
        try:(
            ffmpeg.input(input, ss=start_audio, to=end_audio).output(output, vn=None, ac=1, ar=16000, sample_fmt = "s16").run(cmd=path_ffmpeg)
        )    
        except:
            print('Erro no ffmpeg')

In [24]:
name_audio = "1 - Preparations.aac"
path_in = "/home/elton/Projetos/Translater_Video/teste/resultado_teste"
path_out_2 = "/home/elton/Projetos/Translater_Video/teste/resultado_teste/audios"


In [31]:
picotar_audio(path_in, path_out_2, name_audio, lista)

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

## aaa
python dev

In [10]:
import collections
import contextlib
import sys
import wave
import webrtcvad



In [11]:
def vad_collector_log(sample_rate, frame_duration_ms,
                      padding_duration_ms, vad, frames):
    """
    Igual ao vad_collector, mas retorna uma lista de (inicio, fim)
    indicando os trechos de fala.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    triggered = False

    voiced_frames = []
    timestamps = []  # aqui vamos guardar os logs (inicio, fim)

    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])

            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                start_time = ring_buffer[0][0].timestamp  # início da fala
                voiced_frames.extend([f for f, s in ring_buffer])
                ring_buffer.clear()
        else:
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])

            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                end_time = frame.timestamp + frame.duration  # fim da fala
                timestamps.append((start_time, end_time))
                triggered = False
                ring_buffer.clear()
                voiced_frames = []

    if triggered:
        # caso o áudio acabe no meio de uma fala
        end_time = frame.timestamp + frame.duration
        timestamps.append((start_time, end_time))

    return timestamps


In [ ]:
# def vad_collector_log(sample_rate, frame_duration_ms,
#                       padding_duration_ms, vad, frames):,

